# imports

In [1]:
    import pandas as pd
    import requests
    import numpy as np
    import time
    import bs4
    from bs4 import BeautifulSoup
    import selenium
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.support.select import Select
    from selenium.webdriver.common.action_chains import ActionChains
    

# population page extraction

In [2]:
def extract_population(page,extract_page_catagory_name):
     # and returns it
    time.sleep(5)
    anti_exception=True
    error_counter=0
    while anti_exception==True and error_counter<5: #anti program failsafe 
        try:
            temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
            anti_exception=False #will pass the temp and not get caught when 
        except:
            print("failed to retrieve page element error number:", error_counter)
            error_counter=error_counter+1
            
            
    col_names = temp.find_elements(By.TAG_NAME, "th")
    col_names_list = ["סמל יישוב","שם יישוב"]
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    col_names_list[6]=col_names_list[6] + "זכר" #columns are seperatred between female and male
    col_names_list[7]=col_names_list[7] + "זכר" #its location on the page is mixed with some columns names
    col_names_list[8]=col_names_list[8] + "זכר" #which made it problemetic to extract 
    col_names_list[9]=col_names_list[9] + "נקבה"
    col_names_list[10]=col_names_list[10] + "נקבה"
    col_names_list[11]=col_names_list[11] + "נקבה"
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    table_len = len(rows)  #
    for row in rows[2:]:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    return df.copy()

# construction page extraction

In [15]:
def extract_page_construction(page, extract_page_catagory_name):
    time.sleep(3)
    temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
    col_names = temp.find_elements(By.TAG_NAME, "th")
    col_names_list = []
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    for row in rows[2:]:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    return df.copy()

# general page extraction

In [33]:
def extract_page(page, extract_page_catagory_name):  # creates new dataframe (including columns names) from the page
    # and returns it
    print("extract_page:", extract_page_catagory_name)
    if(extract_page_catagory_name=="construction"):
        return extract_page_construction(page, extract_page_catagory_name)
    if extract_page_catagory_name in ["information", "transportation", "construction", "social_economic index"]:  # columns location differ between catagories
        time.sleep(3)
        temp = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/thead/tr")
        col_names = temp.find_elements(By.TAG_NAME, "th")
    if(extract_page_catagory_name=="population"):
        print("in population")
        return extract_population(page,extract_page_catagory_name)
    col_names_list = []
    for col_name in col_names:  # get columns names
        col_names_list.append(col_name.text)
    df = pd.DataFrame(columns=col_names_list)
    table = page.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody")  # get table
    rows = table.find_elements(By.TAG_NAME, "tr")  # extract each row from a table
    table_len = len(rows)
    for row in rows:
        elements = row.find_elements(By.TAG_NAME, "td")  # extract each element from a row
        elements_text = []
        for i in elements:  # go through each cell in a row (as element not text)
            elements_text.append(i.text)  # convert element which is 1 cell in a row to text
        print(elements_text)
        df.loc[
            len(df)] = elements_text  # adds a new line to the end of data frame len(df)return the length of the dataframe
    
    return df.copy()

# sub function to control past year population pages

In [43]:
def extract_population_by_year(driver,year):#name pending
    time.sleep(3)
    #from here we extract population
    for i in range(3):#move catagories untill population catagory is reached
        time.sleep(2)
        catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
        catagory_menu.click()
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        
    for index in range(2021-year):#change the year to 2018
        time.sleep(1)
        year_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[4]/div")
        year_menu.click()
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
    pop=get_catagory(driver, 1, "population")# extract dataframe and send back
    
    
    return pop

# catagory retrievel function

In [36]:
def get_catagory(pages, page_number, catagory_name="None"): # default values
    print(catagory_name)
    catagory_dataframe = pd.DataFrame()
    time.sleep(3)
    click_array = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[5]/pager/div")  # contains
    # the pages and some more clickable
    pages = click_array.find_elements(By.TAG_NAME, "li")#collect all clickable elements that are pages in 1 list
    if page_number == 1:  # starting condition
        catagory_dataframe = extract_page(driver, catagory_name).copy()
        page_number = page_number + 1 
    for page in pages:#go through the element list that contains all elements that are pages
        if page.text == str(page_number):
            print("scanning page:", page_number)
            page.click() #click the page so its loads in selenium
            catagory_dataframe = pd.concat([catagory_dataframe, extract_page(driver, catagory_name)],
                                           ignore_index=True).copy() #sends the page that is loaded to other function to extract the info from it and link it to the previous dataframe
            page_number = page_number + 1
        if(page.get_attribute("class")=="nextPages"): #this one works removed for since second method is easier on the eye\n",
                page.click()
                catagory_dataframe=pd.concat([catagory_dataframe, get_catagory(driver,page_number,catagory_name)]) 
    return catagory_dataframe.copy()



# main function 2017 catagory extraction

In [37]:

url = 'https://boardsgenerator.cbs.gov.il/pages/webparts/yishuvimpage.aspx'

PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
dataframe_catagory_list = []
target_year = 2018
catagory_names = ["information", "transportation", "construction", "population", "health", "mifkad2008",
                  "data_from_local_authorities", "salary", "welfare", "education", "water", "waste",
                  "social_economic index"]#the catagories in the page

i = 0
for category in catagory_names:#goes through all catagories in the list in order
    anti_exception=True
    error_counter=0
    print(category)
    if(category not in ["information", "transportation", "construction", "population","social_economic index"]):#the catagory is not what we wanted
        catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
        catagory_menu.click()
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        continue
    time.sleep(3)
    print("Scanning: ", category)
    actions = ActionChains(driver)
    if(category !="construction"):
        while anti_exception==True and error_counter<5: #anti crash failsafe 
            try:
                temp = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[4]/table/tbody/tr[2]")
                year_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[4]/div")
                anti_exception=False #if failed loading the element it wont reach here,it will reach only when no error was found loading the element
            except:
                print("failed to retrieve year menu element error number:", error_counter)
                error_counter=error_counter+1
    if category == "social_economic index":#change the year to 2017
        year_menu.click()
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        time.sleep(3)
    elif category == "salary":#change the year to 2017
        for index in range(2):
            year_menu.click()
            actions.send_keys(Keys.ARROW_DOWN)
            actions.send_keys(Keys.ENTER)
            actions.perform()
            time.sleep(3)
    if category == "information" or category == "transportation" or category == "population" :
        for index in range(2022 - target_year):#change the year to 2017
            year_menu.click()
            actions.send_keys(Keys.ARROW_DOWN)
            actions.send_keys(Keys.ENTER)
            actions.perform()
            time.sleep(3)
    dataframe_catagory_list.append(
        get_catagory(driver, 53, category))  # extract the all the pages in the catagory and send the catagory name 
    #and add to a list of dataframes
    #path = "C:\\Users\\omri\\Desktop\\" #C:\Users\ONE1\Desktop\omri
    try:#instead of changing each time,try to save to all 3 known locations
        dataframe_catagory_list[i].to_csv("C:\\Users\\omri\\Desktop\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig') #my main pc
    except:
        try:
            dataframe_catagory_list[i].to_csv("C:\\Users\\ONE1\\Desktop\\omri\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig')#my laptop
        except:
            try:
                dataframe_catagory_list[i].to_csv("C:\\Users\\ONE1\\Desktop\\omri\\" + category + ".csv", index=False,
                                      encoding='utf-8-sig') # nir change this to where you want to save
            except:
                raise # raise error if cant save
          
        #delete below this is a test
    catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
    catagory_menu.click()
    actions = ActionChains(driver)
    actions.send_keys(Keys.ARROW_DOWN)
    actions.send_keys(Keys.ENTER)
    actions.perform()
    i = i + 1
driver.quit()
time.sleep(5)
modular_population_extractor()

information
Scanning:  information
information
information
information
information
information
information
scanning page: 53
extract_page: information
['465', 'צובה', 'ירושלים', 'ירושלים', 'קיבוצים', 'מטה יהודה', 'הרי יהודה', '628', '628', '614', '-', '']
['1136', 'צוחר', 'הדרום', 'באר שבע', 'יישובים כפריים אחרים יהודיים', 'אשכול', 'אזור בשור', '443', '442', '421', '2', '']
['1111', 'צופייה', 'המרכז', 'רחובות', 'מקום', 'חבל יבנה', 'אזור רחובות', '-', '-', '-', '-', '']
['3791', 'צופים', 'יהודה והשומרון', '', 'יישובים יהודיים 4999-2000 תושבים', 'שומרון', '', '2,188', '2,177', '2,123', '11', '']
['198', 'צופית', 'המרכז', 'פתח תקווה', 'מושבים - לרבות כפרים שיתופיים', 'דרום השרון', 'דרום השרון', '1,294', '1,293', '1,275', '1', '']
['1150', 'צופר', 'הדרום', 'באר שבע', 'מושבים - לרבות כפרים שיתופיים', 'הערבה התיכונה', 'הערבה', '419', '418', '412', '1', '']
['1102', 'צוקי ים', 'המרכז', 'השרון', 'יישובים כפריים אחרים יהודיים', 'עמק חפר', 'מערב השרון', '281', '281', '273', '-', '']
['1262', 'צו

['1334', 'ראס אל-עין', 'הצפון', 'עכו', 'יישובים כפריים אחרים לא יהודיים', 'משגב', 'אזור כרמיאל', '362', '-', '-', '362', '']
['990', 'ראס עלי', 'חיפה', 'חיפה', 'יישובים כפריים אחרים לא יהודיים', 'זבולון', 'אזור חיפה', '596', '-', '-', '596', '']
['2640', 'ראש העין', 'המרכז', 'פתח תקווה', 'יישובים יהודיים 99999-50000 תושבים', 'עירייה', 'אזור פתח תקווה', '50,453', '50,429', '49,601', '24', '']
['26', 'ראש פינה', 'הצפון', 'צפת', 'יישובים יהודיים 4999-2000 תושבים', 'מועצה מקומית', 'אזור חצור', '3,061', '3,024', '2,956', '37', '']
['3602', 'ראש צורים', 'יהודה והשומרון', '', 'קיבוצים', 'גוש עציון', '', '934', '934', '934', '-', '']
['8300', 'ראשון לציון', 'המרכז', 'רחובות', 'יישובים יהודיים 499999-200000 תושבים', 'עירייה', 'אזור ראשון לציון', '249,860', '249,681', '229,305', '178', '']
['3795', 'רבבה', 'יהודה והשומרון', '', 'יישובים יהודיים 4999-2000 תושבים', 'שומרון', '', '2,389', '2,389', '2,389', '-', '']
['564', 'רבדים', 'הדרום', 'אשקלון', 'קיבוצים', 'יואב', 'אזור מלאכי', '702', '702', '

['284', 'שדה ורבורג', 'המרכז', 'פתח תקווה', 'מושבים - לרבות כפרים שיתופיים', 'דרום השרון', 'דרום השרון', '1,489', '1,489', '1,471', '-', '']
['293', 'שדה יואב', 'הדרום', 'אשקלון', 'קיבוצים', 'יואב', 'אזור מלאכי', '462', '461', '444', '-', '']
['142', 'שדה יעקב', 'הצפון', 'יזרעאל', 'מושבים - לרבות כפרים שיתופיים', 'עמק יזרעאל', 'אזור יקנעם', '1,100', '1,100', '1,098', '-', '']
['2008', 'שדה יצחק', 'חיפה', 'חדרה', 'מושבים - לרבות כפרים שיתופיים', 'מנשה', 'אזור חדרה', '667', '663', '631', '4', '']
['18', 'שדה משה', 'הדרום', 'אשקלון', 'מושבים - לרבות כפרים שיתופיים', 'לכיש', 'אזור לכיש', '765', '764', '758', '1', '']
['259', 'שדה נחום', 'הצפון', 'יזרעאל', 'קיבוצים', 'עמק המעיינות', 'עמק בית שאן', '681', '681', '665', '-', '']
['329', 'שדה נחמיה', 'הצפון', 'צפת', 'קיבוצים', 'הגליל העליון', 'עמק חולה', '1,215', '1,214', '1,183', '1', '']
['1058', 'שדה ניצן', 'הדרום', 'באר שבע', 'מושבים - לרבות כפרים שיתופיים', 'אשכול', 'אזור בשור', '337', '337', '334', '-', '']
['739', 'שדה עוזיהו', 'הדרום',

['398', 'שרשרת', 'הדרום', 'באר שבע', 'מושבים - לרבות כפרים שיתופיים', 'שדות נגב', 'אזור גרר', '296', '296', '296', '-', '']
['1045', 'שתולה', 'הצפון', 'עכו', 'מושבים - לרבות כפרים שיתופיים', 'מעלה יוסף', 'אזור אילון', '280', '280', '278', '-', '']
['763', 'שתולים', 'הדרום', 'אשקלון', 'מושבים 4999-2000 תושבים', 'באר טוביה', 'אזור מלאכי', '2,002', '2,001', '1,983', '-', '']
['2062', 'תאשור', 'הדרום', 'באר שבע', 'מושבים - לרבות כפרים שיתופיים', 'בני שמעון', 'אזור גרר', '429', '429', '420', '-', '']
scanning page: 63
extract_page: information
['2061', 'תדהר', 'הדרום', 'באר שבע', 'מושבים - לרבות כפרים שיתופיים', 'בני שמעון', 'אזור גרר', '615', '615', '607', '-', '']
['1172', 'תובל', 'הצפון', 'עכו', 'קיבוצים', 'משגב', 'אזור יחיעם', '315', '312', '293', '2', '']
['3558', 'תומר', 'יהודה והשומרון', '', 'מושבים - לרבות כפרים שיתופיים', 'ערבות הירדן', '', '266', '266', '264', '-', '']
['1083', 'תושייה', 'הדרום', 'באר שבע', 'יישובים כפריים אחרים יהודיים', 'שדות נגב', 'אזור גרר', '1,005', '1,005', 

['רחוב', '854', '262', '161', '91', '20', '22', '49', '5', '-', '3', '-', '2']
['רכסים', '922', '1707', '1490', '154', '125', '24', '5', '1', '3', '43', '-', '16']
['רומת הייב', '997', '666', '566', '89', '67', '2', '20', '2', '5', '2', '1', '1']
['רם-און', '1069', '447', '364', '66', '53', '9', '4', '-', '-', '17', '-', '-']
['רהט', '1161', '14533', '10213', '3563', '2259', '448', '856', '39', '13', '56', '16', '633']
['רוויה', '2016', '169', '129', '35', '22', '5', '8', '-', '-', '5', '-', '-']
['רווחה', '2051', '275', '213', '50', '16', '15', '19', '2', '-', '2', '-', '8']
['רימונים', '3565', '275', '235', '25', '15', '8', '2', '-', '5', '9', '-', '1']
['ריחן', '3568', '108', '93', '12', '10', '2', '-', '-', '-', '3', '-', '-']
['רועי', '3619', '93', '66', '24', '18', '6', '-', '-', '1', '2', '-', '-']
['רותם', '3782', '57', '44', '12', '10', '2', '-', '-', '-', '-', '-', '1']
['רחלים', '3822', '171', '156', '15', '15', '-', '-', '-', '-', '-', '-', '-']
['רחובות', '8400', '49158', 

['שרונה', '292', '290', '235', '46', '32', '8', '6', '-', '-', '8', '1', '-']
['שרשרת', '398', '144', '105', '30', '17', '9', '4', '5', '-', '1', '-', '3']
['שעב', '538', '2486', '2072', '359', '293', '52', '14', '-', '3', '27', '-', '25']
['שער אפרים', '661', '834', '710', '86', '68', '15', '3', '-', '2', '36', '-', '-']
['שפיר', '692', '385', '299', '73', '42', '19', '12', '5', '1', '7', '-', '-']
['שתולים', '763', '806', '718', '60', '42', '6', '12', '1', '12', '15', '-', '-']
['שפר', '846', '202', '158', '38', '18', '5', '15', '-', '1', '5', '-', '-']
['שעלבים', '856', '344', '312', '29', '27', '2', '-', '-', '-', '3', '-', '-']
['שער מנשה', '921', '1', '1', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['שתולה', '1045', '156', '127', '21', '17', '1', '3', '-', '-', '8', '-', '-']
['שריגים (לי-און)', '1114', '470', '399', '53', '44', '6', '3', '-', '1', '17', '-', '-']
['שקף', '1233', '203', '152', '42', '26', '13', '3', '-', '1', '8', '-', '-']
['שקד', '3649', '486', '408', '54', '

ElementClickInterceptedException: Message: element click intercepted: Element <div class="select2-container select2-container-active ng-isolate-scope" id="s2id_autogen7" style="width: 75px;">...</div> is not clickable at point (463, 71). Other element would receive the click: <div class="loader ui-loader-background" style="text-align:center; position:fixed;top: 0;left: 0;z-index: 999;width: 100%;">...</div>
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x011237D3]
	(No symbol) [0x010B8B81]
	(No symbol) [0x00FBB36D]
	(No symbol) [0x00FF4E3B]
	(No symbol) [0x00FF26DB]
	(No symbol) [0x00FEFD0B]
	(No symbol) [0x00FEE4D8]
	(No symbol) [0x00FE3253]
	(No symbol) [0x0100B41C]
	(No symbol) [0x00FE2B96]
	(No symbol) [0x0100B774]
	(No symbol) [0x01021215]
	(No symbol) [0x0100B216]
	(No symbol) [0x00FE0D97]
	(No symbol) [0x00FE253D]
	GetHandleVerifier [0x0139ABF2+2510930]
	GetHandleVerifier [0x013C8EC1+2700065]
	GetHandleVerifier [0x013CC86C+2714828]
	GetHandleVerifi

# population 2013-2016 extraction


In [44]:
  #  def modular_population_extractor():
url = 'https://boardsgenerator.cbs.gov.il/pages/webparts/yishuvimpage.aspx'
path = 'C:\Program Files (x86)\chromedriver.exe'
year_array=[2018,2017,2016,2015,2014,2013] #add year 
file_path='C:\\Users\\ONE1\\Desktop\\omri\\'  #C:\Users\ONE1\Desktop\omri
for pop_year in year_array:
    print("loading year:",pop_year)
    time.sleep(3)
    driver = webdriver.Chrome()
    driver.get(url)
    pop_dataframe=extract_population_by_year(driver,pop_year)
    pop_dataframe.to_csv(file_path + "pop" + str(pop_year) + ".csv", index=False,encoding='utf-8-sig')
    driver.quit()


loading year: 2018
population
extract_page: population
in population
extract_page: population
col names are: ['סמל יישוב', 'שם יישוב', 'סה"כ', 'יהודים', 'יהודים ואחרים', 'ערבים', 'סה"כזכר', 'יהודיםזכר', 'יהודים ואחריםזכר', 'ערביםנקבה', 'סה"כנקבה', 'יהודיםנקבה', 'יהודים ואחרים', 'ערבים']
['175', 'אביחיל', '1,410', '', '', '', '694', '', '', '', '716', '', '', '']
['182', 'אבן יהודה', '13,700', '', '', '', '6,723', '', '', '', '6,978', '', '', '']
['400', 'אבן שמואל', '1,675', '', '', '', '882', '', '', '', '793', '', '', '']
['679', 'אביאל', '815', '', '', '', '407', '', '', '', '408', '', '', '']
['783', 'אבן ספיר', '724', '', '', '', '396', '', '', '', '328', '', '', '']
['819', 'אביגדור', '821', '', '', '', '398', '', '', '', '422', '', '', '']
['1,070', 'אביעזר', '865', '', '', '', '436', '', '', '', '429', '', '', '']
['1,081', 'אבן מנחם', '297', '', '', '', '157', '', '', '', '140', '', '', '']
['1,115', 'אביבים', '477', '', '', '', '234', '', '', '', '242', '', '', '']
['1,220', 

['1,125', 'אלמגור', '389', '', '', '', '196', '', '', '', '193', '', '', '']
['1,248', 'אליפז', '151', '', '', '', '73', '', '', '', '79', '', '', '']
['1,253', 'אמנון', '319', '', '', '', '150', '', '', '', '169', '', '', '']
['1,309', 'אלעד', '47,865', '', '', '', '24,292', '', '', '', '23,574', '', '', '']
['3,556', 'אלמוג', '252', '', '', '', '125', '', '', '', '127', '', '', '']
['3,560', 'אלקנה', '3,812', '', '', '', '1,835', '', '', '', '1,977', '', '', '']
['3,618', 'אלעזר', '2,510', '', '', '', '1,219', '', '', '', '1,291', '', '', '']
['3,750', 'אלפי מנשה', '7,866', '', '', '', '3,945', '', '', '', '3,920', '', '', '']
['3,754', 'אספר', '835', '', '', '', '475', '', '', '', '360', '', '', '']
['4,002', 'אלי-עד', '400', '', '', '', '207', '', '', '', '194', '', '', '']
['4,012', 'אניעם', '531', '', '', '', '268', '', '', '', '263', '', '', '']
['529', 'אעבלין', '13,128', '', '', '', '6,619', '', '', '', '6,509', '', '', '']
scanning page: 6
extract_page: population
in populati

['2,033', 'בית חלקיה', '1,221', '', '', '', '817', '', '', '', '404', '', '', '']
['3,575', 'בית חורון', '1,362', '', '', '', '693', '', '', '', '669', '', '', '']
scanning page: 10
extract_page: population
in population
extract_page: population
col names are: ['סמל יישוב', 'שם יישוב', 'סה"כ', 'יהודים', 'יהודים ואחרים', 'ערבים', 'סה"כזכר', 'יהודיםזכר', 'יהודים ואחריםזכר', 'ערביםנקבה', 'סה"כנקבה', 'יהודיםנקבה', 'יהודים ואחרים', 'ערבים']
['94', 'בלפוריה', '513', '', '', '', '275', '', '', '', '238', '', '', '']
['248', 'בית שערים', '744', '', '', '', '348', '', '', '', '397', '', '', '']
['252', 'ביתן אהרן', '121', '', '', '', '62', '', '', '', '60', '', '', '']
['365', 'בית קשת', '707', '', '', '', '339', '', '', '', '368', '', '', '']
['386', 'בני דרור', '1,221', '', '', '', '532', '', '', '', '689', '', '', '']
['592', 'בני דרום', '847', '', '', '', '414', '', '', '', '433', '', '', '']
['598', 'בית קמה', '1,396', '', '', '', '696', '', '', '', '700', '', '', '']
['712', 'בן עמי', '77

['342', 'גברעם', '470', '', '', '', '250', '', '', '', '221', '', '', '']
['370', 'גזר', '285', '', '', '', '137', '', '', '', '149', '', '', '']
['442', 'גדעונה', '458', '', '', '', '230', '', '', '', '228', '', '', '']
['457', 'גזית', '725', '', '', '', '349', '', '', '', '376', '', '', '']
['706', 'גיאה', '970', '', '', '', '493', '', '', '', '478', '', '', '']
['736', 'גילת', '1,437', '', '', '', '737', '', '', '', '700', '', '', '']
['755', 'גורן', '429', '', '', '', '225', '', '', '', '203', '', '', '']
['793', 'גבעתי', '1,022', '', '', '', '502', '', '', '', '520', '', '', '']
['852', 'גונן', '370', '', '', '', '178', '', '', '', '192', '', '', '']
['1,043', 'גיזו', '205', '', '', '', '107', '', '', '', '98', '', '', '']
['1,204', 'גילון', '1,029', '', '', '', '526', '', '', '', '504', '', '', '']
['1,219', 'גורנות הגליל', '186', '', '', '', '92', '', '', '', '94', '', '', '']
['2,550', 'גדרה', '28,093', '', '', '', '13,598', '', '', '', '14,495', '', '', '']
['6,300', 'גבעתיים'

['1,241', 'דקל', '341', '', '', '', '167', '', '', '', '174', '', '', '']
['1,322', 'הודיות', '316', '', '', '', '157', '', '', '', '159', '', '', '']
['9,700', 'הוד השרון', '62,325', '', '', '', '30,353', '', '', '', '31,971', '', '', '']
['1,349', "דריג'את", '1,200', '', '', '', '617', '', '', '', '583', '', '', '']
scanning page: 19
extract_page: population
in population
extract_page: population
col names are: ['סמל יישוב', 'שם יישוב', 'סה"כ', 'יהודים', 'יהודים ואחרים', 'ערבים', 'סה"כזכר', 'יהודיםזכר', 'יהודים ואחריםזכר', 'ערביםנקבה', 'סה"כנקבה', 'יהודיםנקבה', 'יהודים ואחרים', 'ערבים']
['44', 'זוהר', '359', '', '', '', '194', '', '', '', '164', '', '', '']
['464', 'הראל', '269', '', '', '', '136', '', '', '', '132', '', '', '']
['584', 'זיקים', '810', '', '', '', '423', '', '', '', '387', '', '', '']
['788', 'זיתן', '1,081', '', '', '', '550', '', '', '', '531', '', '', '']
['799', 'זכריה', '1,108', '', '', '', '535', '', '', '', '573', '', '', '']
['815', 'זבדיאל', '527', '', '', '

['358', 'יד מרדכי', '767', '', '', '', '398', '', '', '', '369', '', '', '']
['577', 'יד בנימין', '4,175', '', '', '', '2,093', '', '', '', '2,083', '', '', '']
['758', 'יד חנה', '926', '', '', '', '474', '', '', '', '452', '', '', '']
['775', 'יד נתן', '638', '', '', '', '334', '', '', '', '304', '', '', '']
['798', 'יגל', '929', '', '', '', '458', '', '', '', '471', '', '', '']
['1,134', 'יד השמונה', '302', '', '', '', '141', '', '', '', '161', '', '', '']
['1,144', 'ידידה', '212', '', '', '', '154', '', '', '', '58', '', '', '']
['1,177', 'טללים', '509', '', '', '', '244', '', '', '', '265', '', '', '']
['1,181', 'טל-אל', '1,174', '', '', '', '609', '', '', '', '565', '', '', '']
['1,214', 'טפחות', '484', '', '', '', '283', '', '', '', '201', '', '', '']
['1,232', 'יבול', '340', '', '', '', '175', '', '', '', '166', '', '', '']
['2,660', 'יבנה', '46,705', '', '', '', '23,261', '', '', '', '23,444', '', '', '']
['3,743', 'טנא', '847', '', '', '', '447', '', '', '', '400', '', '', '']

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x011237D3]
	(No symbol) [0x010B8B81]
	(No symbol) [0x00FBB36D]
	(No symbol) [0x00FAD403]
	(No symbol) [0x00FAD241]
	(No symbol) [0x00FABC95]
	(No symbol) [0x00FAC63A]
	(No symbol) [0x00FC45AE]
	(No symbol) [0x00FBDBC0]
	(No symbol) [0x00FBD703]
	(No symbol) [0x00FBDF6C]
	(No symbol) [0x00FBE250]
	(No symbol) [0x00FE8950]
	(No symbol) [0x0100B41C]
	(No symbol) [0x00FE2B96]
	(No symbol) [0x0100B774]
	(No symbol) [0x01021215]
	(No symbol) [0x0100B216]
	(No symbol) [0x00FE0D97]
	(No symbol) [0x00FE253D]
	GetHandleVerifier [0x0139ABF2+2510930]
	GetHandleVerifier [0x013C8EC1+2700065]
	GetHandleVerifier [0x013CC86C+2714828]
	GetHandleVerifier [0x011D3480+645344]
	(No symbol) [0x010C0FD2]
	(No symbol) [0x010C6C68]
	(No symbol) [0x010C6D4B]
	(No symbol) [0

    if category not in ["construction"]: #You can add categories in this list  removed the not so it runs on all
        time.sleep(3)
        catagory_menu = driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div/div/div[2]/span[2]/div")
        catagory_menu.click()
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        continue